# Apache Mesos HTTP API

This notebook will show how to connect to a running Master/Slave and launch commands via a simple `CommandInfo` protocol buffer.

The main goal of this notebook is to show how to interact with the new [Mesos HTTP API](https://github.com/apache/mesos/blob/master/docs/scheduler_http_api.md) in Python.

## Prerequisites

- you have RTFM (link above);
- you know how to build/run Apache Mesos locally (see the [Starting Guide](http://mesos.apache.org/gettingstarted/))
- you are familiar with Python [Requests](http://www.python-requests.org/en/latest/) framework.

## Starting Mesos

Nothing unusal here, start ZooKeeper (`zkServer.sh start`) then start Master/Slave:
```
cd /path/to/mesos/build
make -j 4 V=0

# Optional, but recommended:
make -j 4 V=0 check

# If all tests pass:
./bin/mesos-master.sh --zk=zk://localhost:2181/mesos/test --work_dir=/tmp/mesos-24 --quorum=1 --port=5051           

.... lots of logging here

# In another shell:
./bin/mesos-slave.sh --master=zk://localhost:2181/mesos/test --work_dir=/tmp/slave --port=5055
```

Then navigate to the [Mesos Web UI](http://localhost:5051) and make sure all it's working just fine.

If the above doesn't work, it's unlikely that anything in the following ever will.

## Python Virtualenv

I always strongly recommend that folks use virtual environments when messing around with Python and installing libraries - feel free to skip this, but if you end up borking your system... **you have been warned**.

Create a `requirements.txt` file with the following contents:
```
doctools==0.2.2
docutils==0.12
ipython==3.1.0
itsdangerous==0.24
Jinja2==2.7.3
pyzmq==14.6.0
requests==2.7.0
sh==1.11
simplejson==3.6.5
six==1.9.0
stevedore==1.4.0
tornado==4.1
urllib3==1.10.4
virtualenv==12.1.1
virtualenv-clone==0.2.5
virtualenvwrapper==4.5.0
Werkzeug==0.10.4
```

My actual `dev` virtualenv has a lot more stuff, but the above should be sufficient to get you going (and probably need strictly even less than that - YMMV); most of the dependencies above are for IPython Notebooks.

```
mkvirtualenv dev
pip install -r requirements.txt
ipython notebook
```
Then load this file in your Notebook.

Happy hacking!


# Common Imports & Useful globals

In [1]:
# We need to use the magic function %px on every engine when we run in Cluster mode.
# See below the note about executing multi-threaded code.
from __future__ import print_function

import json
import os
import requests
import sh
from threading import Thread
import time


SUBSCRIBE_BODY = {
    "type": "SUBSCRIBE",
    "subscribe": {
        "framework_info": {
            "user" :  "marco",
            "name" :  "Example HTTP Framework"
        },
        "force" : True
    }
}

TEARDOWN_BODY = {
    "framework_id": {
        "value" : None
    },
    "type": "TEARDOWN"
}


# Adjust the ports according to how you launched Mesos:
# see --port in the commands in "Prerequisites"
MASTER_URL = 'http://localhost:5051'
SLAVE_URL = 'http://localhost:5055'
API_V1 = '/api/v1/scheduler'
API_URL = '{}/{}'.format(MASTER_URL, API_V1)
CONTENT = 'application/json'

headers = {
    "Content-Type": CONTENT, 
    "Accept": CONTENT, 
    "Connection": "close"
}


# TODO: THIS IS THREAD-UNSAFE
terminate = False
offers = []
accepted_offer = False
framework_id = None

## POST helper method

This will also be useful when we will need to run on separate kernels.

In [2]:
def post(url, body, **kwargs):
    print('Connecting to Master: ' + url)
    r = requests.post(url, headers=headers, data=json.dumps(body), **kwargs)
    
    if r.status_code not in [200, 202]:
        raise ValueError("Error sending request: {} - {}".format(r.status_code, r.text))
    if 'stream' in kwargs:
        # The streaming format needs some munging:
        first_line = True
        for line in r.iter_lines():
            if first_line:
                count_bytes = int(line)
                first_line = False
                continue
            body = json.loads(line[:count_bytes])
            count_bytes = int(line[count_bytes:])
            if body.get("type") == "HEARTBEAT":
                continue
            # When we get OFFERS we want to see them (and eventually, use them)
            if body.get("type") == "OFFERS" and not accepted_offer:
                print(body.get("offers"))
                global offers
                offers = body.get("offers")
            elif accepted_offer:
                # TODO: must send a DECLINE message
                pass
                
            # We need to capture the framework_id to use in subsequent requests.
            if body.get("type") == "SUBSCRIBED":
                global framework_id
                framework_id = body.get("subscribed").get("framework_id").get("value")
                if framework_id:
                    print("Framework {} registered with Master at ({})".format(framework_id, url))
            if terminate:
                break

## Warm up

The following code just checks that there is connectivity and the settings are all correct: do not move forward until this run successfully.

In [3]:
r = requests.get("{}/state.json".format(MASTER_URL))
master_state = r.json()
print("Mesos version running at {}".format(master_state["version"]))

r = requests.get("{}/state.json".format(SLAVE_URL))
slave_state = r.json()

# If this is not true, you're in for a world of hurt:
assert master_state["version"] == slave_state["version"]

def get_framework(index=None, id=None):
    if index and id:
        raise ValueError("Cannot specify both ID and Index")
    r = requests.get("{}/state.json".format(MASTER_URL))
    master_state = r.json()
    frameworks = master_state.get("frameworks")
    if frameworks and isinstance(frameworks, list):
        if index is not None and len(frameworks) > index:
            return frameworks[index]
        elif id:
            for framework in frameworks:
                if framework.get("id") == id:
                    return framework

# And right now there ought to be no frameworks:
assert get_framework(index=0) is None

Mesos version running at 0.25.0


# Registering a Framework

Using the HTTP API requires to run at least two separate threads: one for the "incoming" Master messages **to** the Framework (the HTTP connection we opened with the initial `SUBSCRIBE` `POST`) and another **from** the Framework to the Master to actual convey our requests (eg, accepting `OFFER`s).

We will be using the `threading` module, as this is I/O-bound and there is no CPU contention; we will run a background thread (`persistent_channel`) to receive messages from Mesos, and will use the main thread to send `requests` to Master.

The code in this Notebook **is not thread-safe**; in particular, we don't use any form of locking, as there is no real concern about races over shared data: in real production code, one should obviously protect shared data with suitable `locks` (see the [Python Multithreading documentation](https://docs.python.org/3/library/threading.html) for more details).

In [4]:
try:
    kwargs = {'stream':True, 'timeout':30}
    persistent_channel = Thread(target=post, args=(API_URL, SUBSCRIBE_BODY), kwargs=kwargs)
    persistent_channel.daemon = True
    persistent_channel.start()
    
    # We need to retrieve the framework_id:
#     while not framework_id:
#         time.sleep(3)
    
    print("Framework ID: {}".format(framework_id))
except Exception, ex:
    print("An error occurred: {}".format(ex))

Framework ID: None
Connecting to Master: http://localhost:5051//api/v1/scheduler


# Terminating a Framework

The request above will keep running forever (but see [Terminating the Request](#terminating) below) until we tear down the framework we just started:

In [41]:
framework = get_framework(0)
if framework:
    fid = framework['id']
    body = TEARDOWN_BODY
    body['framework_id']['value'] = fid
    print(body, json.dumps(body))
    print(post(API_URL, body))
else:
    print("No frameworks to terminate")

No frameworks to terminate


## <a name="terminating"></a>Terminating the Request

In [50]:
if persistent_channel.is_alive():
    terminate = True

In [5]:
persistent_channel.is_alive(), terminate

{u'framework_id': {u'value': u'20150820-224310-855746752-5051-17686-0005'}}


(True, False)

# Accepting Offers for Resources

We need a tiny amount of resources (0.1 CPU, 32 MB of RAM) to run a simple command on the Slave.

In [6]:
print(framework_id)

{u'offers': [{u'url': {u'path': u'/slave(1)', u'scheme': u'http', u'address': {u'ip': u'192.168.1.51', u'hostname': u'gondor', u'port': 5055}}, u'hostname': u'gondor', u'framework_id': {u'value': u'20150820-224310-855746752-5051-17686-0005'}, u'agent_id': {u'value': u'20150820-125856-855746752-5051-6674-S0'}, u'id': {u'value': u'20150820-224310-855746752-5051-17686-O4'}, u'resources': [{u'type': u'SCALAR', u'scalar': {u'value': 12}, u'role': u'*', u'name': u'cpus'}, {u'type': u'SCALAR', u'scalar': {u'value': 31094}, u'role': u'*', u'name': u'mem'}, {u'type': u'SCALAR', u'scalar': {u'value': 431720}, u'role': u'*', u'name': u'disk'}, {u'ranges': {u'range': [{u'begin': 31000, u'end': 32000}]}, u'type': u'RANGES', u'role': u'*', u'name': u'ports'}]}]}
20150820-224310-855746752-5051-17686-0005


In [7]:
print(offers)

{u'offers': [{u'url': {u'path': u'/slave(1)', u'scheme': u'http', u'address': {u'ip': u'192.168.1.51', u'hostname': u'gondor', u'port': 5055}}, u'hostname': u'gondor', u'framework_id': {u'value': u'20150820-224310-855746752-5051-17686-0005'}, u'agent_id': {u'value': u'20150820-125856-855746752-5051-6674-S0'}, u'id': {u'value': u'20150820-224310-855746752-5051-17686-O4'}, u'resources': [{u'type': u'SCALAR', u'scalar': {u'value': 12}, u'role': u'*', u'name': u'cpus'}, {u'type': u'SCALAR', u'scalar': {u'value': 31094}, u'role': u'*', u'name': u'mem'}, {u'type': u'SCALAR', u'scalar': {u'value': 431720}, u'role': u'*', u'name': u'disk'}, {u'ranges': {u'range': [{u'begin': 31000, u'end': 32000}]}, u'type': u'RANGES', u'role': u'*', u'name': u'ports'}]}]}
